In [24]:
import pandas as pd
import numpy as np
import os
from weclapp_api import create_article

In [3]:
def detect_encoding(file_path):
    # Try reading the file with different encodings
    encodings = ['utf-8', 'latin1', 'cp1252', 'windows-1252', 'ISO-8859-1']  # Add more encodings as necessary

    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            return encoding
        
        except UnicodeDecodeError:
            continue  # Try next encoding if current one fails

    # If all encodings fail
    raise ValueError("Could not determine the encoding of the CSV file")

In [4]:
def handle_columns(df):
    
    try: 
        df.insert(0,"Kopfartikelnummer",np.nan)
        df.insert(1,"Kopfartikelname",np.nan)
        df.insert(2,"Kopfartikelbeschreibung",np.nan)
        df.insert(3,"Artikelname",np.nan)
        df.insert(4,"Artikelbeschreibung",np.nan)
        df.insert(5,"Positionsnummer",np.nan)
        df.rename(columns={"Bezeichnung":"Artikelnummer","Menge":"Anzahl"},inplace=True)
    except Exception as e:
        print(e)

In [5]:
def get_grid_type_and_colour(df_grouped):
    dummy = df_grouped[df_grouped["Artikelnummer"].str.startswith(("G.", "GF"))]
    
    typ = dummy.iloc[0]["Artikelnummer"].split(".")[0]

    if dummy.iloc[0]["Artikelnummer"].split(".")[1] == "NB":
        colour = "NB"
    elif dummy.iloc[0]["Artikelnummer"].split(".")[1] == "NG":
        colour = "NG"
    elif dummy.iloc[0]["Artikelnummer"].split(".")[1] == "NW":
        colour = "NW"
    
    else:
        print("coudn't recognize colour")
    
    return typ,colour

In [6]:
def solve_sm4_case():
    # test SM4
    if not artikelliste_df_grouped[artikelliste_df_grouped["Artikelnummer"].str.startswith("SM4")].empty:
        
        dummy_grouped = artikelliste_df_grouped[artikelliste_df_grouped["Artikelnummer"].str.startswith(("G.", "GF"))]

        # test existing Grids and their colours 
        if artikelliste_df_grouped[artikelliste_df_grouped["Artikelnummer"].str.startswith(("G.", "GF"))].shape[0] > 1:
            print("The Cubes in this configuration do have different colours or are mixed with flame retardned and normal. The special case about SM4 cannot be solved now. Be aware.")
        
        elif (dummy_grouped["Anzahl"] <=2).any():
            print("Zu wenige Grids in Liste. SM4 cant be solved. Be aware!")
        
        else: 

            # normale Grids minus 2 
            artikelliste_df_grouped.loc[artikelliste_df_grouped["Artikelnummer"].str.startswith(("G.", "GF")), "Anzahl"] -= 2

            # spezielle Grid + 1 je nach farbe
            typ, colour = get_grid_type_and_colour(artikelliste_df_grouped)
            artikelnummer = typ + "." + colour + "_SM4"
            artikelliste_df_grouped.loc[len(artikelliste_df_grouped)]  = {"Kopfartikelnummer":kopfartikelnummer, "Artikelnummer":artikelnummer , "Anzahl":1}

In [7]:
# loop through csvs, insert /delete the right columns and insert kopfartikelnummer

for file in os.listdir():
    if file.endswith(".csv"):
        
        kopfartikelnummer = file.split(".")[0]
        encoding = detect_encoding(file)
        
        try:
            artikelliste_df = pd.read_csv(file, sep=";", encoding=encoding)
        except Exception as e:
            print(e)
        
        # get rid of empty Strings
        artikelliste_df = artikelliste_df.apply(lambda x: x.replace("", None))
        
        #query and group
        artikelliste_df.query("Pos.notnull()", inplace=True)
        artikelliste_df_grouped = artikelliste_df.groupby("Bezeichnung")["Menge"].sum().reset_index()
        
        handle_columns(artikelliste_df_grouped)
        
        artikelliste_df_grouped["Kopfartikelnummer"] = kopfartikelnummer

        try:
            solve_sm4_case()

        except Exception as e:
            print(e)


        artikelliste_df_grouped.to_csv(f"finished_{kopfartikelnummer}.csv", index=False, sep=";", encoding="ISO-8859-1")

In [25]:
article_number = "teste alles außer Liste"
article_name = input("Wie soll der Artikel heißen?")
create_article(article_number,article_name)

Fehler beim Erstellen von Artikel teste alles außer Liste
Status Code: 400 Content: b'{\n  "detail": "articleNumber is duplicate",\n  "error": "articleNumber is duplicate",\n  "status": 400,\n  "title": "validation failed",\n  "type": "/webapp/view/api/errors.html#!/errors/validation",\n  "validationErrors": []\n}'


In [12]:
import requests

In [22]:

url = "https://degridstudio.weclapp.com/webapp/api/v1/article?sort=-lastModifiedDate"
token = os.getenv("weclapp_token")
headers = {"AuthenticationToken" : token,
            "Content-Type" : "application/json"}


re = requests.get(url=url,headers=headers)
re = re.json()
re["result"]

[{'id': '807068',
  'version': '0',
  'active': True,
  'applyCashDiscount': True,
  'articleAlternativeQuantities': [],
  'articleCalculationPrices': [],
  'articleImages': [],
  'articleNumber': 'spezialanfertigungen',
  'articlePrices': [],
  'articleType': 'BASIC',
  'availableForSalesChannels': [],
  'availableInSale': True,
  'batchNumberRequired': False,
  'billOfMaterialPartDeliveryPossible': False,
  'commissionRate': '100',
  'createdDate': 1721220698228,
  'customAttributes': [{'attributeDefinitionId': '338023',
    'booleanValue': False}],
  'customerArticleNumbers': [],
  'defaultPriceCalculationType': 'PURCHASE',
  'defaultStoragePlaces': [],
  'defineIndividualTaskTemplates': False,
  'lastModifiedDate': 1721220698227,
  'lowLevelCode': 0,
  'marginCalculationPriceType': 'PURCHASE_PRICE_PRODUCTION_COST',
  'name': 'testerei',
  'packagingUnitBaseArticleId': '807068',
  'priceCalculationParameters': [],
  'productionArticle': False,
  'productionBillOfMaterialItems': [],


In [ ]:
# articleCategoryId': '48583'


#   'articleType': 'SALES_BILL_OF_MATERIAL',
#   'availableForSalesChannels': [],
#   'availableInSale': True,
#   'batchNumberRequired': False,
#   'billOfMaterialPartDeliveryPossible': True,


#   'salesBillOfMaterialItems': [{'id': '805694',
#     'version': '0',
#     'articleId': '82463',
#     'articleNumber': 'G.NG',
#     'createdDate': 1721119433142,
#     'lastModifiedDate': 1721119433141,
#     'positionNumber': 1,
#     'quantity': '24'},
#    {'id': '805695',
#     'version': '0',
#     'articleId': '111653',
#     'articleNumber': 'G.NG_PB',
#     'createdDate': 1721119433143,
#     'lastModifiedDate': 1721119433142,
#     'positionNumber': 2,
#     'quantity': '6'},

In [67]:
input("Press Enter to finish the script")

''